# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - Kyber/Crystalis

Implementar o esquema Kyber/Crystalis em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento Kyber/Crystalis (https://pq-crystals.org/kyber/data/kyber-specification-round3.pdf)

### Kyber/Crystalis - KEM-IND-CPA

In [184]:
import math, os

'''
Parâmetros definidos para a implementação do kyber
'''
n = 256
_n = 9
q = 3329
Qq = PolynomialRing(GF(q), 'x')
y = Qq.gen()
RQ = QuotientRing(Qq, y^n+1)

'''
Definição da função mod+-
'''
def modMm(r,a) :
    _r = r % a
    # Testar se a é par
    if mod(a,2)==0 :
        # Cálculo dos limites -a/2 e a/2
        inf_bound, sup_bound = -a/2, a/2
    # a é ímpar
    else :
        # Cálculo dos limites -a-1/2 e a-1/2
        inf_bound, sup_bound = (-a-1)/2, (a-1)/2
    # Queremos garantir que o módulo se encontre no intervalo calculado
    while _r > sup_bound :
        _r-=a
    while _r < inf_bound :
        _r+=a
    return _r
    
    

'''
Função que converte um array de bytes num array de bits
'''
def bytesToBits(bytearr) :
    bitarr = []
    for elem in bytearr :
        bitElemArr = []
        # Calculamos cada bit pertencente ao byte respetivo
        for i in range(0,8) :
            bitElemArr.append(mod(elem // 2**(mod(i,8)),2))
        
        for i in range(0,len(bitElemArr)) :
            bitarr.append(bitElemArr[i])
    return bitarr

'''
Função que converte um array de bits num array de bytes
'''
def bitsToBytes(bitarr) :
    bytearr = []
    bit_arr_size = len(bitarr)
    byte_arr_size = bit_arr_size / 8
    for i in range(byte_arr_size) :
        elem = 0
        for j in range(8) : # Definir macro BYTE_SIZE = 0
            elem += (int(bitarr[i*8+j]) * 2**j)
        bytearr.append(elem)
    return bytearr
    
        


''' 
Função parse cuja finalidade é receber como input 
uma byte stream e retornar, como output, a representação NTT
'''
def parse(stream) :
    coefs = []
    i,j = 0,0
    while j<n :
        d1 = stream[i] + 256*(mod(stream[i+1],16))
        d2 = stream[i+1]//16 + 16*stream[i+2]
        if d1<q :
            coefs.append(d1)
            j+=1
        elif d2<q and j<n :
            coefs.append(d2)
            j+=1
        i+=3
    return RQ(coefs)

'''
Definição da função compress
'''
def compress(q,x,d) :
    return mod(round((2**d)/q * x),2**d)

'''
Definição da função decompress
'''
def decompress(q,x,d) :
    return round((q/2**d) * x)


'''
Definição da função CBD. Recebe como 
input o n (comprido) e o array de bytes
'''
def cbd(noise, btarray) :
    f = []
    bitArray = bytesToBits(btarray)
    for i in range(256) :
        a, b = 0, 0
        # Cálculo do a e do b
        for j in range(256) :
            a+=bitArray[2*i*noise + j]
            b+=bitArray[2*i*noise + noise + j]
        f.append(a - b)
    return RQ(f)


'''
Implementação da função decode
'''
def decode(l, btarray) :
    f = []
    bitArray = bytesToBits(btarray)
    for i in range(256) :
        fi = 0
        for j in range(l) :
            fi += int(bitArray[i*l+j]) * 2**j
        f.append(fi)
    return RQ(f)

'''
Implementação da função encode
'''
def encode(l, poly) :
    bitArr = []
    coef_array = poly.list()
    # Percorremos cada coeficiente
    for i in range(n) :
        actual = int(coef_array[i])
        for j in range(l) :
            bitArr.append(actual % 2)
            actual = actual // 2
    return bitsToBytes(bitArr)


In [182]:
arr = [32,42,34,5,35,3,5,7,54,34,21,43,5,2,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,0,0,0,32,42,34,5,35,3,5,7,54,34,21,43,5,2,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,0,0,0]
poly = decode(2,arr)
#print(len(poly.list()))
#print(poly.list())
#print(len(encode(2,poly)))
print(arr)
print(encode(2,poly))
if arr == encode(2,poly) :
    print('\n[Success] Os arrays correspondem!')
else :
    print('\n[Erro] Algo correu mal com o decode/encode!')

[32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[Success] Os arrays correspondem!


In [57]:
'''
Implementação da classe NTT
'''
class NTT(object):
#    
    def __init__(self, n=256):
        if not any([n == t for t in [32,64,128,256,512,1024,2048]]):
            raise ValueError("improper argument ",n)
        self.n = n  
        self.q = 2*n+1
        while True:
            if (self.q).is_prime():
                break
            self.q += 2*n
            
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen(); self.w = w
        
        g = (w^n + 1)
        print(len(g.roots(multiplicities=False)))
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)

In [73]:
# Teste

T = NTT()

f = T.random_pol(64)
# print(f)

ff = T.ntt(f)

fff = T.ntt_inv(ff)

# print(fff)
print("Correto ? ",f == fff)

256
Correto ?  True


In [ ]:
'''
Implementação da classe 
'''

class KyberPKE :
    
    def __init__(self,n,k,q,n1,n2,du,dv) :
        
        self.n = n
        self.k = k
        self.q = q
        self.n1 = n1
        self.n2 = n2
        self.du = du
        self.dv = dv
        Qq = PolynomialRing(GF(q), 'x')
        y = Qq.gen()
        RQ = QuotientRing(Qq, y^n+1)
        self.Rq = RQ
        
    '''
    Função que permite a geração de 
    uma chave pública
    '''
    def keygen(sk) :
        d = os.urandom(32)